In [1]:
import requests
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from power.ml_ops.data import get_pv_data, clean_pv_data
from power.ml_ops.model import model_yesterday
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
data = get_pv_data()
clean_data =clean_pv_data(data)

# data loaded
# data cleaned


In [3]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376944 entries, 0 to 376943
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   utc_time     376944 non-null  datetime64[ns, UTC]
 1   local_time   376944 non-null  object             
 2   electricity  376944 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 8.6+ MB


In [ ]:
data = get_pv_data()
# data_clean = clean_pv_data(data)
n_years = 5
reduced_idx = 365 * 24 * n_years
df = data.iloc[-reduced_idx:,:].reset_index()

# Converting dates to datetime objects
df['local_time'] = pd.to_datetime(df['local_time'], utc=True)

# Setting dates as the index
df.set_index(['local_time'], inplace=True)

df

In [ ]:
def compress(df, **kwargs):
    """    Reduces size of dataframe by downcasting numerical columns    """
    input_size = df.memory_usage(index=True).sum()/ 1024
    print("new dataframe size: ", round(input_size,2), 'kB')

    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")

    return df

compress(df)

In [ ]:
# Additive Decomposition (y = Trend + Seasonal + Residuals)
result_add = seasonal_decompose(df['electricity'], model='additive')
result_add.plot();

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(1,2, figsize=(16,3))

plot_acf(df.electricity, lags=200, ax=axes[0]);
plot_pacf(df.electricity, lags=200, ax=axes[1], color='r');